In [1]:
from bs4 import BeautifulSoup
import requests
import urllib
import json 

In [2]:
def get_onion_image_url(link):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    r = requests.get(link, headers=hdr)
    data = r.text  #Get the website source as text
    soup = BeautifulSoup(data) #Setup a "soup" which BeautifulSoup can search
    vals =  soup.findAll(attrs={"name":"twitter:image"})
    for v in vals:
        if v["content"] not in ("https://x.kinja-static.com/assets/images/logos/placeholders/theonion.png", ):
            return v["content"]

In [3]:
def get_hp_image_url(link):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    r = requests.get(link, headers=hdr)
    data = r.text  #Get the website source as text
    soup = BeautifulSoup(data) #Setup a "soup" which BeautifulSoup can search
    vals = soup.findAll(attrs={"property":"og:image:url"})
    for v in vals:
        if v["content"] not in ("https://img.huffingtonpost.com/asset/default-entry.jpg?ops=1778_1000"):
            return v["content"]

In [4]:
with open('Sarcasm_Headlines_Dataset_v2.json', 'r') as handle:
    json_data = [json.loads(line) for line in handle]
    for i, d in enumerate(json_data):
        try:
            link = d["article_link"]
            if d["is_sarcastic"] == 1:
                image_link = get_onion_image_url(link)
                urllib.request.urlretrieve(image_link, "satire_images/" + str(i) + ".jpg")
            else:
                image_link = get_hp_image_url(link)
                urllib.request.urlretrieve(image_link, "serious_images/" + str(i) + ".jpg")
            json_data[i]["has_image"] = True
        except:
            json_data[i]["has_image"] = False

with open("dataset_with_images.json", "w") as outfile: 
    json.dump(json_data, outfile)
    